In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
from torch.utils.data import random_split, DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import v2
from pathlib import Path
import wandb

import sys

sys.path.append("../src")

from trainer import Trainer
from module import VGGNetModule
from utils import model_size
from callbacks import OverfitCallback
from logger import WandbLogger

In [ ]:
data_path = Path("../data")
logs_path = Path("../logs")

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [ ]:
logger = WandbLogger(
    project_name="ImageClassification",
    config={
        "model_architecture": "ConvNet",
        "batch_size": 64,
        "epochs": 1,
        "optimizer": {
            "name": "Adam",
        },
        "train_split": 42_000,
        "val_split": 8000
    },
    logs_path=logs_path
)

In [ ]:
dataset = CIFAR10(data_path, train=True, download=True, transform=v2.Compose([
    # Normalize
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010))
]))

train_dataset, val_dataset = random_split(
    dataset, [logger.config["train_split"], logger.config["val_split"]])

train_dataloader = DataLoader(
    train_dataset, batch_size=logger.config["batch_size"], shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=logger.config["batch_size"])

Files already downloaded and verified


In [ ]:
callbacks = [
    OverfitCallback(limit_batches=2)
]

In [ ]:
module = VGGNetModule()

optimizer = torch.optim.Adam(
    params=module.model.parameters(),
)

trainer = Trainer(
    module=module,
    logger=logger,
    optimizer=optimizer,
    callbacks=callbacks,
    logs_path=logs_path,
    device=device,
)

In [9]:
try:
    trainer.fit(train_dataloader, val_dataloader)
except KeyboardInterrupt as e:
    print("Run stopped!")
finally:
    wandb.finish()

ValueError: Key values passed to `wandb.log` must be strings.

[Metrics](https://api.wandb.ai/links/sampath017/iwrrziwg)